# MNIST Syft Data Owner

In [ ]:
import syft as sy
sy.LOG_FILE = "syft_do.log"
_ = sy.logger.add(sy.LOG_FILE, enqueue=True, colorize=False, diagnose=True, backtrace=True, level="TRACE")

# PART 1: Launch a Duet Server and Connect

As a data owner, you want to allow someone else to perform data science on data that you own and likely want to protect.

In order to do this, we must load our data into a locally running server (within this notebook).
We call this server a "Duet".

To begin, you must launch Duet and help your Duet "partner" (a data scientist) connect to this server.

You do this by running the code below and following the instructions it gives!

In [ ]:
duet = sy.launch_duet(loopback=True)

In [ ]:
# accept all incoming requests, print and log them
accept_handler = {
    "request_name": "age_data",
    "timeout_secs": -1,
    "action": "accept",
    "print_local": True,
    "log_local": True
}
duet.requests.add_handler(accept_handler)
accept_handler = {
    "request_name": "cuda_is_available",
    "timeout_secs": -1,
    "action": "accept",
    "print_local": True,
    "log_local": True
}
duet.requests.add_handler(accept_handler)
accept_handler = {
    "request_name": "loss",
    "timeout_secs": -1,
    "action": "deny",
    "print_local": True,
    "log_local": True
}
duet.requests.add_handler(accept_handler)
accept_handler = {
    "request_name": "train_size",
    "timeout_secs": -1,
    "action": "accept",
    "print_local": True,
    "log_local": True
}
duet.requests.add_handler(accept_handler)
accept_handler = {
    "request_name": "inference",
    "timeout_secs": -1,
    "action": "accept",
    "print_local": True,
    "log_local": True
}
duet.requests.add_handler(accept_handler)
accept_handler = {
    "request_name": "model_download",
    "timeout_secs": -1,
    "action": "accept",
    "print_local": True,
    "log_local": True
}
duet.requests.add_handler(accept_handler)

In [ ]:
duet.requests.handlers

In [ ]:
for handler in duet.requests.handlers:
    del handler["created_time"]
    duet.requests.remove_handler(handler)

In [ ]:
# deny all incoming requests, print and log them
deny_handler = {
    "timeout_secs": -1,
    "action": "deny",
    "print_local": True,
    "log_local": True
}
duet.requests.add_handler(deny_handler)

In [ ]:
# # accept all incoming requests, print and log them
# accept_handler = {
#     "timeout_secs": -1,
#     "action": "accept",
#     "print_local": True,
#     "log_local": True
# }
# duet.requests.add_handler(accept_handler)

If you are in Jupyter Notebook (not Colab) the ☝🏾DUET LIVE STATUS above will animate 

# PART 2: Launch a Duet Server and Connect

In [ ]:
import torch as th
import torchvision as tv

In [ ]:
# # let's pretend we have a tensor of human ages
# age_data = th.rand((9216, 128)) # not working too big??
# import sys
# sys.getsizeof(age_data)

In [ ]:
# age_data = th.rand((9216, 64)) # works Wall time: 19.4 s
# age_data = th.rand((9216, 96)) # works Wall time: 46.6 s
# age_data = th.rand((4608, 128))   # 589824 params # works Wall time: 24.7 s
age_data = th.rand((10, 1))   # params # works Wall time: 24.7 s
age_data_pointer = age_data.send(duet, searchable=True)

# Let's tag it as "age data"
age_data = age_data.tag("#age_data")

# Let's add a description to the tensor
age_data = age_data.describe("This is a list of people's ages. Let's keep it private!")

# Let's UPLOAD THE DATA to the Duet server!
age_data_pointer = age_data.send(duet, searchable=True)

In [ ]:
duet.requests.remove_handler(accept_handler)

In [ ]:
# accept all incoming requests, print and log them
deny_handler = {
    "timeout_secs": 10,
    "action": "deny",
    "print_local": True,
    "log_local": True
}
duet.requests.add_handler(deny_handler)

In [ ]:
duet.requests.remove_handler(deny_handler)

In [ ]:
duet.requests.pandas